# (Optional) How to solve OOV problem?

## How did you solve this problem in your programming task?

用基本事件代替，也就是默认有一个。

平滑方法总结：
<p>http://www.shuang0420.com/2017/03/24/NLP%20%E7%AC%94%E8%AE%B0%20-%20%E5%B9%B3%E6%BB%91%E6%96%B9%E6%B3%95(Smoothing)%E5%B0%8F%E7%BB%93/

## Read about the 'Turing-Good Estimator', can you explain the main points about this method, and may implement this method in your programming task?

### 下面看一个例子：

想法：(Things seen once) 使用刚才已经看过一次的事物的数量来帮助估计从来没有见过的事物的数量。
举个例子，假设你在钓鱼，然后抓到了 18 条鱼，种类如下：10 carp, 3 perch, 2 whitefish, 1 trout, 1 salmon, 1 eel。

**那么下一条鱼是 trout 的概率是多少？**
很简单，我们认为是 1/18.

**那么，下一条鱼是新品种的概率是多少？**
不考虑其他，那么概率是 0，然而根据 Things seen once 来估计新事物，概率是 3/18.

**在此基础上，下一条鱼是 trout 的概率是多少？**
肯定就小于 1/18，那么怎么估计呢？
在 Good-Turing 下，对每一个计数 $r$，我们做一个调整，变为 $r^*$，公式如下，其中 $N_r$ 表示出现过 $r$ 次的鱼。

$$
r^{*}=(r+1) \frac{N_{r+1}}{N_{r}}.
$$

然后，我们就有
$$
P_{G T}(x : c(x)=r)=\frac{r^{*}}{N}
$$

所以，$c=1$ 时，
$$
\begin{array}{l}{C^{*}(\text {trou} t)=2 * 1 / 3=2 / 3}, \\[6pt] 
{P^{*}(\text {trou} t)=\frac{2 / 3}{18}=\frac{1}{27}}.
\end{array}
$$

然后，问题来了，如果 $N_{r+1} = 0$ 怎么办？这在 $r$ 很高的情况下很常见，因为在对计数进行计数时(counts of counts)，会出现 “holes”。即使没有这个 hole，对很高的 $r$ 来说，$N_r$ 也是有噪音的。

所以，我们应该这样来看 $r$:
$$
r^{*}=(r+1) \frac{E\left(n_{r+1}\right)}{E\left(n_{r}\right)}.
$$
但是，问题是怎么估计这种期望呢？这需要更多的解释。

**Jelinek-Mercer smoothing(Interpolation, 差值)**

同样，以语言模型为例，我们看这样一种情况，如果 $c(BURNISH THE)$ 和 $c(BURNISH THOU)$ 都是 0，那么在前面的 Good-Turing 里，

$$p(THE|BURNISH)=p(THOU|BURNISH),$$

而这个概率我们直观上来看是错误的，因为 THE 要比 THOU 常见的多，

$$
p(T H E | B U R N I S H)>p(T H O U | B U R N I S H)
$$ 

应该是大概率事件。要实现这个，我们就希望把 bigram 和 unigram 结合起来，interpolate 就是这样一种方法。


用线性差值把不同阶的 N-gram 结合起来，这里结合了 trigram，bigram 和 unigram。用 lambda 进行加权：

$$
\begin{aligned} p\left(w_{n} | w_{n-1} w_{n-2}\right) &=\lambda_{1} p\left(w_{n} | w_{n-1} w_{n-2}\right) +\lambda_{2} p\left(w_{n} | w_{n-1}\right) +\lambda_{3} p\left(w_{n}\right) \end{aligned},
$$

其中

$$
\sum_{i} \lambda_{i}=1.
$$